In [6]:
from PIL import Image
import glob
import numpy as np

def extract_image_data(file_path):
    extracted_set = []
    for filename in glob.glob(file_path):
        img = Image.open(filename) 
        img = img.resize((100,100))   # reshape to 200 * 200 pixel for model analysis
        data = list(img.getdata())   # extract image to RGB data points
        img.close()
        data = np.array(list(map(list, data))) # convert the data into 3D list
        extracted_set.append(data)
    return extracted_set
        
man_clothing = extract_image_data('Man-Clothing/*.JPEG')
woman_clothing = extract_image_data('Woman-Clothing/*.JPEG')

X = np.array(man_clothing + woman_clothing)
X = X.reshape(400, 100, 100, 3)
Y = np.array([0] * len(man_clothing) + [1] * len(woman_clothing))
Y = Y.reshape(Y.shape[0], )

In [8]:
from keras.applications.vgg16 import VGG16

model = VGG16(weights='imagenet', include_top=False, input_shape=(100,100,3))
features = model.predict(X)

print (features.shape)

(400, 3, 3, 512)


In [30]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers import Conv2D

model = Sequential()
model.add(Dense(32, input_shape=(3, 3, 512), activation='relu'))
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['accuracy'])

model.fit(features, Y, epochs=50, batch_size=10, validation_split=0.3)
model.summary()

scores = model.evaluate(features, Y)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

Train on 280 samples, validate on 120 samples
Epoch 1/50
280/280 [==============================] - 1s 3ms/step - loss: 0.3760 - acc: 0.5786 - val_loss: 0.9625 - val_acc: 0.0167
Epoch 2/50
280/280 [==============================] - 0s 373us/step - loss: 0.2813 - acc: 0.6964 - val_loss: 0.9748 - val_acc: 0.0083
Epoch 3/50
280/280 [==============================] - 0s 399us/step - loss: 0.2832 - acc: 0.6929 - val_loss: 0.9276 - val_acc: 0.0667
Epoch 4/50
280/280 [==============================] - 0s 334us/step - loss: 0.2475 - acc: 0.7250 - val_loss: 0.8942 - val_acc: 0.0833
Epoch 5/50
280/280 [==============================] - 0s 343us/step - loss: 0.2018 - acc: 0.7893 - val_loss: 0.7526 - val_acc: 0.1917
Epoch 6/50
280/280 [==============================] - 0s 319us/step - loss: 0.1992 - acc: 0.7679 - val_loss: 0.5583 - val_acc: 0.3500
Epoch 7/50
280/280 [==============================] - 0s 350us/step - loss: 0.1843 - acc: 0.7857 - val_loss: 0.8230 - val_acc: 0.1333
Epoch 8/50
280/280